In [1]:
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

import tensorflow as tf
import numpy as np

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

#X_train = X_train - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]
#X_test = X_test - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]

X_train_full, y_train_full = X_train, y_train

X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]



c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\ferbi\appd

In [2]:
import tensorflow as tf
import numpy as np
from datetime import datetime
import math

def execute_code(rand_seed = 1, lambda_exp = -6.0):
    LOC = """

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

print("Now:", now)

n_inputs = X_train.shape[1]
n_hidden1 = 1024
n_hidden2 = 1024
n_hidden3 = 1024
n_hidden4 = 1024
n_hidden5 = 1024
n_outputs = 10

def reset_graph(seed=43):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph(@rand_seed@)

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

epsVal = pow(10.0, -8.0)

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2/np.sqrt(n_inputs + n_neurons)
        init = tf.random_normal((n_inputs, n_neurons), mean=0.0, stddev=stddev)
        
        W = tf.Variable(init, name="kernel")
        #W = tf.cast(tf.abs(W) >= epsVal, W.dtype) * W
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.add(tf.matmul(X, W), b)
                
        if activation is not None:
            return activation(Z)
        else:
            return Z
        
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    hidden3 = neuron_layer(hidden2, n_hidden3, name="hidden3", activation=tf.nn.relu)
    hidden4 = neuron_layer(hidden3, n_hidden4, name="hidden4", activation=tf.nn.relu)
    hidden5 = neuron_layer(hidden4, n_hidden5, name="hidden5", activation=tf.nn.relu)
    logits = neuron_layer(hidden5, n_outputs, name="outputs")

hidden1_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden1/kernel:0")
hidden2_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden2/kernel:0")
hidden3_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden3/kernel:0")
hidden4_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden4/kernel:0")
hidden5_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden5/kernel:0")
logits_W = tf.get_default_graph().get_tensor_by_name("dnn/outputs/kernel:0")
    
def calcRegValue(W, type=1, eps=0.00001):
    if type == 1:
        wMax = 1.0
        gt_eps = tf.minimum(1.0, tf.add(1.0, tf.constant(eps/(wMax-eps))*tf.subtract(tf.abs(W), wMax)))
        #gt_eps = 1.0  
        #lt_eps = tf.constant((1-eps)/eps)*tf.abs(W)
        lt_eps = 0.0
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*gt_eps + tf.cast(tf.abs(W) < eps, W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 2:
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*W
        return tf.reduce_sum(reg_value)
    elif type == 3:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.pow(W, 2))))
    elif type == 4:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.abs(W))))
    elif type == 5:
        return tf.reduce_sum(tf.abs(W))
    elif type == 6:
        return tf.reduce_sum(tf.pow(W, 2))
    elif type == 7:
        gt_eps = 1.0  
        epsMat = tf.constant(eps, shape=W.get_shape())
        lt_eps = 1.0 - tf.pow(epsMat, tf.abs(W))
        reg_value = tf.cast(tf.abs(W) >= np.sqrt(eps), W.dtype)*gt_eps + tf.cast(tf.abs(W) < np.sqrt(eps), W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 8:
        eMat = tf.constant(math.e, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(eMat, tf.subtract(0.0, 1000*tf.pow(W, 2)))))
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    regType = 8
    total_regvalue = tf.add_n([calcRegValue(hidden1_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden2_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden3_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden4_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden5_W, type=regType, eps=epsVal), \
                               calcRegValue(logits_W, type=regType, eps=epsVal)])
    
    total_nzw = tf.cast(tf.add_n([tf.count_nonzero(hidden1_W), \
                                  tf.count_nonzero(hidden2_W), \
                                  tf.count_nonzero(hidden3_W), \
                                  tf.count_nonzero(hidden4_W), \
                                  tf.count_nonzero(hidden5_W), \
                                  tf.count_nonzero(logits_W)]), tf.float32)
    total_nzw_eps = tf.cast(tf.add_n([tf.reduce_sum(tf.cast(abs(hidden1_W) < epsVal, hidden1_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden2_W) < epsVal, hidden2_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden3_W) < epsVal, hidden3_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden4_W) < epsVal, hidden4_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden5_W) < epsVal, hidden5_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(logits_W) < epsVal, logits_W.dtype))]), tf.int64)
    
    lambdaParam = tf.pow(10.0, @lambda_exp@, name="lambdaParam")
    loss_total = tf.add(tf.reduce_mean(xentropy, name="loss_total"), tf.scalar_mul(lambdaParam, total_regvalue))
    #loss_total = tf.reduce_mean(xentropy, name="loss_total")
    
learning_rate = tf.placeholder(tf.float32, shape=(), name="learning_rate")
with tf.name_scope("train"):
    #optimizer = tf.train.ProximalGradientDescentOptimizer(learning_rate)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    #optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
    #optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    training_op = optimizer.minimize(loss_total)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

reg_summary = tf.summary.scalar('Count_Reg', total_regvalue)
loss_summary = tf.summary.scalar('Loss_Total', loss_total)
err_summary = tf.summary.scalar('Class_Err_Perc', (1-accuracy)*100)
#file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_epochs = 250
batch_size = 20

def shuffle_batch(X, y, batch_size):
    np.random.seed(43)
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    batch_num = -1
    for batch_idx in np.array_split(rnd_idx, n_batches):
        batch_num = batch_num + 1
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield batch_num, X_batch, y_batch

def get_lr_step(epochCounter, start_lr, stepsize, end_lr, lr_iter_period):
    lr = start_lr - math.floor(epochCounter/lr_iter_period) * stepsize
    return max(lr, end_lr)
        
def get_lr_triangle(epochCounter, lr_min, lr_max, lr_iter_period):
    cycle = math.floor(1 + epochCounter/(2*lr_iter_period))
    x = abs(epochCounter/lr_iter_period - 2*cycle + 1)
    lr = lr_min + (lr_max - lr_min) * max(0, (1-x)/math.pow(2, cycle-1))
    return lr

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

start_time = datetime.now()

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        lr = get_lr_triangle(epoch, lr_min=0.01, lr_max=0.25, lr_iter_period=20)
        #lr=0.01
        rnd_idx = np.random.permutation(len(X_train))
        for rnd_indices in np.array_split(rnd_idx, len(X_train) // batch_size):
            X_batch, y_batch = X_train[rnd_indices], y_train[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, learning_rate: lr})
        
        loss_val, reg_loss_val, lambda1, acc_val = sess.run([loss_total, total_regvalue, lambdaParam, accuracy], \
                                                   feed_dict={X: X_valid, y: y_valid, learning_rate: lr})
        base_loss_val = loss_val - lambda1*reg_loss_val
        
        if best_loss > loss_val:
            save_path = saver.save(sess, "./models/model_" + now + ".ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation losses: {:.4f}, {:.4f}, {:.4f}\tBest Loss: {:.4f} ({})\tAccuracy: {:.2f}%".format(
            epoch, base_loss_val, lambda1*reg_loss_val, loss_val, best_loss, checks_without_progress, acc_val * 100))

end_time = datetime.now()
running_time = end_time - start_time
print("Total running time:\t", running_time.seconds)

with tf.Session() as sess:
    saver.restore(sess, "./models/model_" + now + ".ckpt") # or better, use save_path
    #saver.restore(sess, "./models/model_20190129210937.ckpt") # or better, use save_path
    n_batches = len(X_test) // 2000
    batch_remaining = n_batches
    correct_count = 0.0
    for batch_idx in np.array_split(range(len(X_test)), n_batches):
        correct_count = correct_count + len(batch_idx)*accuracy.eval(feed_dict={X: X_test[batch_idx], y: y_test[batch_idx]})
        print("Batches remaining:    ", batch_remaining)
        batch_remaining = batch_remaining - 1
print("Test error rate:    {:.10f}%".format(100*(1-correct_count/len(X_test)))) 
"""
    LOC = LOC.replace("@rand_seed@", str(rand_seed))
    LOC = LOC.replace("@lambda_exp@", str(lambda_exp))
    exec(LOC)

In [3]:
for j in [-4.0, -4.2, -4.4, -4.6, -4.8, -5.0, -5.2, -5.4, -5.6, -5.8, -6.0, -6.2, -6.4, -6.6, -6.8, -7.0]:
    for i in [1, 2, 3]:
        print("RANDOM SEED:  ", i, "\tLAMBDA_EXP:  ", j)
        execute_code(rand_seed=i, lambda_exp=j)

RANDOM SEED:   1 	LAMBDA_EXP:   -4.0
Now: 20240308222017
0	Validation losses: 0.5488, 99.3560, 99.9048	Best Loss: 99.9048 (0)	Accuracy: 78.02%
1	Validation losses: 0.4336, 59.4475, 59.8810	Best Loss: 59.8810 (0)	Accuracy: 84.94%
2	Validation losses: 0.4365, 44.7957, 45.2322	Best Loss: 45.2322 (0)	Accuracy: 83.82%
3	Validation losses: 0.4602, 36.9921, 37.4523	Best Loss: 37.4523 (0)	Accuracy: 83.36%
4	Validation losses: 0.4212, 32.0930, 32.5142	Best Loss: 32.5142 (0)	Accuracy: 84.50%
5	Validation losses: 0.4648, 28.7382, 29.2030	Best Loss: 29.2030 (0)	Accuracy: 82.76%
6	Validation losses: 0.3540, 26.2556, 26.6096	Best Loss: 26.6096 (0)	Accuracy: 87.10%
7	Validation losses: 0.4225, 24.3809, 24.8033	Best Loss: 24.8033 (0)	Accuracy: 84.46%
8	Validation losses: 0.3966, 22.8798, 23.2764	Best Loss: 23.2764 (0)	Accuracy: 85.70%
9	Validation losses: 0.4779, 21.6663, 22.1442	Best Loss: 22.1442 (0)	Accuracy: 83.78%
10	Validation losses: 0.3599, 20.6327, 20.9926	Best Loss: 20.9926 (0)	Accuracy: 86.

19	Validation losses: 0.3689, 15.7465, 16.1154	Best Loss: 16.1154 (0)	Accuracy: 86.12%
20	Validation losses: 0.4549, 15.4395, 15.8944	Best Loss: 15.8944 (0)	Accuracy: 84.18%
21	Validation losses: 0.3764, 15.1862, 15.5625	Best Loss: 15.5625 (0)	Accuracy: 85.96%
22	Validation losses: 0.3449, 14.9419, 15.2868	Best Loss: 15.2868 (0)	Accuracy: 87.52%
23	Validation losses: 0.3397, 14.7505, 15.0902	Best Loss: 15.0902 (0)	Accuracy: 87.44%
24	Validation losses: 0.3331, 14.5902, 14.9233	Best Loss: 14.9233 (0)	Accuracy: 87.84%
25	Validation losses: 0.3783, 14.4631, 14.8414	Best Loss: 14.8414 (0)	Accuracy: 84.90%
26	Validation losses: 0.3489, 14.3438, 14.6927	Best Loss: 14.6927 (0)	Accuracy: 87.14%
27	Validation losses: 0.3203, 14.2177, 14.5381	Best Loss: 14.5381 (0)	Accuracy: 87.20%
28	Validation losses: 0.3275, 14.0924, 14.4198	Best Loss: 14.4198 (0)	Accuracy: 87.88%
29	Validation losses: 0.3169, 14.0079, 14.3248	Best Loss: 14.3248 (0)	Accuracy: 88.10%
30	Validation losses: 0.3341, 13.9327, 14.2

114	Validation losses: 0.3340, 11.8837, 12.2178	Best Loss: 12.2178 (0)	Accuracy: 89.74%
115	Validation losses: 0.3382, 11.8703, 12.2085	Best Loss: 12.2085 (0)	Accuracy: 89.66%
116	Validation losses: 0.3487, 11.8676, 12.2163	Best Loss: 12.2085 (1)	Accuracy: 88.94%
117	Validation losses: 0.3416, 11.8524, 12.1940	Best Loss: 12.1940 (0)	Accuracy: 89.92%
118	Validation losses: 0.3596, 11.8434, 12.2030	Best Loss: 12.1940 (1)	Accuracy: 89.80%
119	Validation losses: 0.3397, 11.8353, 12.1750	Best Loss: 12.1750 (0)	Accuracy: 89.78%
120	Validation losses: 0.3473, 11.8270, 12.1744	Best Loss: 12.1744 (0)	Accuracy: 90.14%
121	Validation losses: 0.3423, 11.8263, 12.1686	Best Loss: 12.1686 (0)	Accuracy: 89.96%
122	Validation losses: 0.3453, 11.8250, 12.1703	Best Loss: 12.1686 (1)	Accuracy: 89.96%
123	Validation losses: 0.3513, 11.8227, 12.1739	Best Loss: 12.1686 (2)	Accuracy: 89.42%
124	Validation losses: 0.3543, 11.8224, 12.1767	Best Loss: 12.1686 (3)	Accuracy: 89.82%
125	Validation losses: 0.3525, 1

208	Validation losses: 0.4818, 11.3269, 11.8087	Best Loss: 11.7923 (5)	Accuracy: 89.52%
209	Validation losses: 0.4823, 11.3236, 11.8059	Best Loss: 11.7923 (6)	Accuracy: 89.68%
210	Validation losses: 0.4742, 11.3285, 11.8027	Best Loss: 11.7923 (7)	Accuracy: 89.16%
211	Validation losses: 0.4743, 11.3272, 11.8015	Best Loss: 11.7923 (8)	Accuracy: 89.54%
212	Validation losses: 0.4879, 11.3190, 11.8069	Best Loss: 11.7923 (9)	Accuracy: 89.08%
213	Validation losses: 0.4869, 11.3220, 11.8089	Best Loss: 11.7923 (10)	Accuracy: 89.64%
214	Validation losses: 0.4816, 11.3145, 11.7961	Best Loss: 11.7923 (11)	Accuracy: 89.62%
215	Validation losses: 0.4824, 11.3009, 11.7833	Best Loss: 11.7833 (0)	Accuracy: 89.52%
216	Validation losses: 0.4940, 11.3186, 11.8126	Best Loss: 11.7833 (1)	Accuracy: 89.10%
217	Validation losses: 0.5093, 11.3171, 11.8265	Best Loss: 11.7833 (2)	Accuracy: 89.48%
218	Validation losses: 0.4843, 11.3162, 11.8005	Best Loss: 11.7833 (3)	Accuracy: 89.40%
219	Validation losses: 0.5001,

9	Validation losses: 0.3922, 15.7540, 16.1462	Best Loss: 16.1462 (0)	Accuracy: 86.34%
10	Validation losses: 0.3968, 14.9962, 15.3930	Best Loss: 15.3930 (0)	Accuracy: 85.12%
11	Validation losses: 0.4531, 14.3295, 14.7826	Best Loss: 14.7826 (0)	Accuracy: 84.10%
12	Validation losses: 0.3939, 13.7622, 14.1561	Best Loss: 14.1561 (0)	Accuracy: 85.56%
13	Validation losses: 0.3948, 13.2864, 13.6813	Best Loss: 13.6813 (0)	Accuracy: 86.00%
14	Validation losses: 0.4234, 12.8495, 13.2728	Best Loss: 13.2728 (0)	Accuracy: 84.14%
15	Validation losses: 0.3635, 12.4729, 12.8363	Best Loss: 12.8363 (0)	Accuracy: 87.30%
16	Validation losses: 0.3945, 12.4758, 12.8703	Best Loss: 12.8363 (1)	Accuracy: 85.02%
17	Validation losses: 0.3841, 11.9739, 12.3580	Best Loss: 12.3580 (0)	Accuracy: 86.50%
18	Validation losses: 0.3695, 11.6580, 12.0275	Best Loss: 12.0275 (0)	Accuracy: 87.12%
19	Validation losses: 0.3730, 11.4646, 11.8376	Best Loss: 11.8376 (0)	Accuracy: 86.70%
20	Validation losses: 0.3311, 11.1709, 11.50

...

27	Validation losses: 0.3451, 0.4040, 0.7491	Best Loss: 0.7320 (2)	Accuracy: 89.22%
28	Validation losses: 0.3460, 0.3970, 0.7430	Best Loss: 0.7320 (3)	Accuracy: 89.82%
29	Validation losses: 0.3179, 0.3907, 0.7087	Best Loss: 0.7087 (0)	Accuracy: 90.06%
30	Validation losses: 0.3574, 0.3857, 0.7431	Best Loss: 0.7087 (1)	Accuracy: 89.96%
31	Validation losses: 0.3818, 0.3804, 0.7622	Best Loss: 0.7087 (2)	Accuracy: 90.28%
32	Validation losses: 0.3717, 0.3762, 0.7479	Best Loss: 0.7087 (3)	Accuracy: 90.56%
33	Validation losses: 0.3826, 0.3724, 0.7550	Best Loss: 0.7087 (4)	Accuracy: 90.86%
34	Validation losses: 0.4061, 0.3693, 0.7754	Best Loss: 0.7087 (5)	Accuracy: 90.38%
35	Validation losses: 0.4375, 0.3663, 0.8038	Best Loss: 0.7087 (6)	Accuracy: 90.48%
36	Validation losses: 0.4303, 0.3640, 0.7944	Best Loss: 0.7087 (7)	Accuracy: 90.64%
37	Validation losses: 0.4790, 0.3622, 0.8412	Best Loss: 0.7087 (8)	Accuracy: 90.74%
38	Validation losses: 0.5228, 0.3607, 0.8835	Best Loss: 0.7087 (9)	Accuracy:

23	Validation losses: 0.3397, 0.3575, 0.6972	Best Loss: 0.6971 (2)	Accuracy: 89.36%
24	Validation losses: 0.3137, 0.3497, 0.6634	Best Loss: 0.6634 (0)	Accuracy: 89.62%
25	Validation losses: 0.3384, 0.3415, 0.6799	Best Loss: 0.6634 (1)	Accuracy: 89.68%
26	Validation losses: 0.3264, 0.3340, 0.6604	Best Loss: 0.6604 (0)	Accuracy: 90.24%
27	Validation losses: 0.3294, 0.3279, 0.6573	Best Loss: 0.6573 (0)	Accuracy: 90.04%
28	Validation losses: 0.3560, 0.3220, 0.6780	Best Loss: 0.6573 (1)	Accuracy: 90.56%
29	Validation losses: 0.4019, 0.3171, 0.7190	Best Loss: 0.6573 (2)	Accuracy: 89.86%
30	Validation losses: 0.3997, 0.3125, 0.7122	Best Loss: 0.6573 (3)	Accuracy: 90.26%
31	Validation losses: 0.3878, 0.3086, 0.6965	Best Loss: 0.6573 (4)	Accuracy: 90.88%
32	Validation losses: 0.3967, 0.3051, 0.7017	Best Loss: 0.6573 (5)	Accuracy: 90.50%
33	Validation losses: 0.4222, 0.3023, 0.7244	Best Loss: 0.6573 (6)	Accuracy: 90.66%
34	Validation losses: 0.4612, 0.2995, 0.7607	Best Loss: 0.6573 (7)	Accuracy:

15	Validation losses: 0.3471, 0.3376, 0.6847	Best Loss: 0.6568 (1)	Accuracy: 88.72%
16	Validation losses: 0.3286, 0.3293, 0.6579	Best Loss: 0.6568 (2)	Accuracy: 88.80%
17	Validation losses: 0.3190, 0.3214, 0.6404	Best Loss: 0.6404 (0)	Accuracy: 89.20%
18	Validation losses: 0.3271, 0.3142, 0.6413	Best Loss: 0.6404 (1)	Accuracy: 89.32%
19	Validation losses: 0.3556, 0.3082, 0.6638	Best Loss: 0.6404 (2)	Accuracy: 89.08%
20	Validation losses: 0.3357, 0.3015, 0.6372	Best Loss: 0.6372 (0)	Accuracy: 88.98%
21	Validation losses: 0.3175, 0.2950, 0.6125	Best Loss: 0.6125 (0)	Accuracy: 89.58%
22	Validation losses: 0.3361, 0.2891, 0.6252	Best Loss: 0.6125 (1)	Accuracy: 89.80%
23	Validation losses: 0.3046, 0.2829, 0.5875	Best Loss: 0.5875 (0)	Accuracy: 90.36%
24	Validation losses: 0.3485, 0.2773, 0.6258	Best Loss: 0.5875 (1)	Accuracy: 89.28%
25	Validation losses: 0.3446, 0.2724, 0.6170	Best Loss: 0.5875 (2)	Accuracy: 89.24%
26	Validation losses: 0.3897, 0.2675, 0.6572	Best Loss: 0.5875 (3)	Accuracy:

15	Validation losses: 0.3200, 0.3370, 0.6570	Best Loss: 0.6570 (0)	Accuracy: 88.22%
16	Validation losses: 0.3208, 0.3295, 0.6503	Best Loss: 0.6503 (0)	Accuracy: 88.30%
17	Validation losses: 0.2926, 0.3217, 0.6143	Best Loss: 0.6143 (0)	Accuracy: 89.62%
18	Validation losses: 0.3498, 0.3141, 0.6638	Best Loss: 0.6143 (1)	Accuracy: 88.10%
19	Validation losses: 0.3281, 0.3075, 0.6356	Best Loss: 0.6143 (2)	Accuracy: 89.12%
20	Validation losses: 0.3498, 0.3011, 0.6509	Best Loss: 0.6143 (3)	Accuracy: 87.98%
21	Validation losses: 0.3411, 0.2951, 0.6362	Best Loss: 0.6143 (4)	Accuracy: 88.86%
22	Validation losses: 0.3808, 0.2886, 0.6694	Best Loss: 0.6143 (5)	Accuracy: 89.00%
23	Validation losses: 0.3455, 0.2822, 0.6277	Best Loss: 0.6143 (6)	Accuracy: 88.88%
24	Validation losses: 0.3262, 0.2766, 0.6028	Best Loss: 0.6028 (0)	Accuracy: 89.62%
25	Validation losses: 0.3478, 0.2715, 0.6193	Best Loss: 0.6028 (1)	Accuracy: 89.10%
26	Validation losses: 0.3466, 0.2668, 0.6135	Best Loss: 0.6028 (2)	Accuracy:

16	Validation losses: 0.3163, 0.2357, 0.5521	Best Loss: 0.5384 (1)	Accuracy: 89.10%
17	Validation losses: 0.3263, 0.2325, 0.5588	Best Loss: 0.5384 (2)	Accuracy: 88.98%
18	Validation losses: 0.3381, 0.2292, 0.5672	Best Loss: 0.5384 (3)	Accuracy: 88.80%
19	Validation losses: 0.3611, 0.2262, 0.5873	Best Loss: 0.5384 (4)	Accuracy: 88.26%
20	Validation losses: 0.3371, 0.2235, 0.5606	Best Loss: 0.5384 (5)	Accuracy: 88.18%
21	Validation losses: 0.3113, 0.2205, 0.5318	Best Loss: 0.5318 (0)	Accuracy: 89.34%
22	Validation losses: 0.3335, 0.2172, 0.5507	Best Loss: 0.5318 (1)	Accuracy: 89.18%
23	Validation losses: 0.3494, 0.2141, 0.5635	Best Loss: 0.5318 (2)	Accuracy: 89.44%
24	Validation losses: 0.3474, 0.2113, 0.5587	Best Loss: 0.5318 (3)	Accuracy: 89.64%
25	Validation losses: 0.3477, 0.2085, 0.5562	Best Loss: 0.5318 (4)	Accuracy: 89.54%
26	Validation losses: 0.3720, 0.2058, 0.5778	Best Loss: 0.5318 (5)	Accuracy: 90.10%
27	Validation losses: 0.3745, 0.2032, 0.5777	Best Loss: 0.5318 (6)	Accuracy: